In [4]:
import numpy as np
import numba
import timeit


# Original function
def original_find_peaks(signal):
    ds = np.diff(signal, axis=0)
    ds = np.insert(ds, 0, ds[0])
    mask = np.argwhere(np.abs(ds[1:]) <= 1e-3).squeeze()
    ds[mask] = ds[mask - 1]
    ds = np.sign(ds)
    ds = np.diff(ds)
    ds = np.insert(ds, 0, ds[0])
    t = np.argwhere(ds > 0)
    p = np.argwhere(ds < 0)
    return p, t


# Optimized function with numba
@numba.jit(nopython=True)
def optimized_find_peaks(signal):
    n = len(signal)
    ds = np.empty(n, dtype=signal.dtype)
    ds[0] = 0
    for i in range(1, n):
        ds[i] = signal[i] - signal[i - 1]

    mask = np.abs(ds) <= 1e-3
    for i in range(n):
        if mask[i]:
            ds[i] = ds[i - 1]

    for i in range(1, n):
        ds[i] = np.sign(ds[i])
    ds[0] = ds[1]

    changes = np.empty(n - 1, dtype=ds.dtype)
    for i in range(1, n):
        changes[i - 1] = ds[i] - ds[i - 1]

    t = np.where(changes > 0)[0]
    p = np.where(changes < 0)[0]
    return p, t


def time_functions():
    signal = np.random.rand(1000)  # Adjust size as needed

    original_time = timeit.timeit(lambda: original_find_peaks(signal), number=100)
    optimized_time = timeit.timeit(lambda: optimized_find_peaks(signal), number=100)

    print(f"Original function time: {original_time:.5f} seconds")
    print(f"Optimized function time: {optimized_time:.5f} seconds")


time_functions()

Original function time: 0.00744 seconds
Optimized function time: 0.22901 seconds
